In [1]:
from binance.client import Client
import pandas as pd
import datetime

In [2]:
api_key = None
api_secret = None

In [53]:
def get_timerange_klines(api_key, api_secret, symbol, start_time, end_time, interval='second'):
    """
    根据 symbol 和 time 查询一段时间内的 K 线数据，
    返回该 K 线的收盘价 (close price)。

    参数:
      symbol: 交易对标识，如 "BTCUSDT"
      start_time: 目标时间（datetime 对象）
      end_time: 目标时间（datetime 对象）
      interval: 间隔，必须从['second','minute','hour']中选择
      api_key: 币安的API Key
      api_secret: 币安的API Secret

    返回:
      最近 K 线的收盘价格 (float)。如果没有数据，则返回 None。
    """
    
    client = Client(api_key, api_secret)
    if interval == 'second':
        klines = client.get_historical_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1SECOND, start_str=str(start_time), end_str=str(end_time))
    elif interval == 'minute':
        klines = client.get_historical_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE, start_str=str(start_time), end_str=str(end_time))
    elif interval == 'hour':
        klines = client.get_historical_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1HOUR, start_str=str(start_time), end_str=str(end_time))
    else:
        raise Exception("interval should be selected from ['second', 'minute', 'hour']")
        
    df_M = pd.DataFrame(klines, columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
    columns_to_convert = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']

    for col in columns_to_convert:
        df_M[col] = df_M[col].astype(float)

    return df_M

In [62]:
def get_closest_price(api_key, api_secret, symbol, time):
    """
    根据 symbol 和 time 查询最近的 1 秒钟 K 线数据，
    返回该 K 线的收盘价 (close price)。

    参数:
      symbol: 交易对标识，如 "BTCUSDT"
      time: 目标时间（datetime 对象）
      api_key: 币安的API Key
      api_secret: 币安的API Secret

    返回:
      最近 K 线的收盘价格 (float)。如果没有数据，则返回 None。
    """
    
    start_time = time
    end_time = start_time + datetime.timedelta(seconds=1)
    df_M = get_timerange_klines(api_key, api_secret, symbol, start_time, end_time)
    try:
        price = df_M['Close'][0]
    except:
        price = None
    return price
    

In [89]:
def get_interval_volumn(api_key, api_secret, symbol, time, interval_type, interval_amount):
    """
    根据 symbol 和 time 查询time前后相应时间的volumn总值。

    参数:
      symbol: 交易对标识，如 "BTCUSDT"
      time: 目标时间（datetime 对象）
      interval_type: 间隔类型，必须从['second','minute','hour']中选择
      interval_amount: 间隔数量，非零整数
      api_key: 币安的API Key
      api_secret: 币安的API Secret

    返回:
      最近 K 线的收盘价格 (float)。如果没有数据，则返回 None。
    """
    
    if interval_amount > 0:
        start_time = time
    else:
        end_time = time
    
    if interval_type == 'second':
        if interval_amount > 0:
            start_time = time
            end_time = start_time + datetime.timedelta(seconds=int(interval_amount))
        else:
            end_time = time
            start_time = end_time + datetime.timedelta(seconds=int(interval_amount))
    elif interval_type == 'minute':
        if interval_amount > 0:
            start_time = time
            end_time = start_time + datetime.timedelta(minutes=int(interval_amount))
        else:
            end_time = time
            start_time = end_time + datetime.timedelta(minutes=int(interval_amount))
    elif interval_type == 'hour':
        if interval_amount > 0:
            start_time = time
            end_time = start_time + datetime.timedelta(hours=int(interval_amount))
        else:
            end_time = time
            start_time = end_time + datetime.timedelta(hours=int(interval_amount))
    else:
        raise Exception("interval_type should be selected from ['second', 'minute', 'hour']")

    df_M = get_timerange_klines(api_key, api_secret, symbol, start_time, end_time, interval=interval_type)

    #     print(df_M)
    
    try:
        volumn = df_M['Volume'].sum()
        
    except:
        volumn = None
        
    return volumn
    

In [90]:
symbol = 'BTCUSDT'
time = datetime.datetime(2025, 3, 15, 0, 0, 0)
get_closest_price(api_key, api_secret, symbol, time)

83983.2

In [93]:
interval_type = 'minute'
interval_amount = 5
get_interval_volumn(api_key, api_secret, symbol, time, interval_type, interval_amount)

       Open Time      Open      High       Low     Close    Volume  \
0  1741996800000  83983.19  84052.93  83983.19  84045.49  21.71669   
1  1741996860000  84045.49  84045.49  83964.57  83971.29   7.41994   
2  1741996920000  83971.29  84059.18  83971.29  84048.15   6.58345   
3  1741996980000  84048.15  84075.12  84042.42  84042.42   5.68202   
4  1741997040000  84042.42  84063.19  84021.07  84063.18   5.89182   
5  1741997100000  84063.18  84063.19  84035.56  84053.96   5.27675   

      Close Time  Quote Asset Volume  Number of Trades  \
0  1741996859999        1.824733e+06            2993.0   
1  1741996919999        6.232609e+05            1804.0   
2  1741996979999        5.531481e+05            1356.0   
3  1741997039999        4.776263e+05             905.0   
4  1741997099999        4.952126e+05            1117.0   
5  1741997159999        4.435294e+05            1408.0   

   Taker Buy Base Asset Volume  Taker Buy Quote Asset Volume Ignore  
0                     10.49778  

52.57067000000001